# Testing supervised classifiers with output of Fmask

In [41]:
import numpy as np
import rasterio
from scipy.stats import mode
from sklearn.feature_extraction.image import extract_patches_2d
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

Create a set of patches from an S2 image and from an output of Fmask.

To prevent having to read all bands of the image into memory at once, we set the random state to ensure the same patches are extracted from each band.

In [34]:
%%timeit -n 1 -r 1
patches = []
seed = 101
with rasterio.open('/data/mbp14mtp/devon/devon.tif') as src:
    for idx in src.indexes:
        img = src.read(idx)
        p = extract_patches_2d(img, (3, 3),
                               max_patches=20000,
                               random_state=seed)
        patches.append(p.reshape((20000, 9)))
patches = np.stack(patches, axis=-1)

1 loop, best of 1: 1min 27s per loop


Find the median of the 3x3 pixel patch

In [35]:
patch_medians = np.median(patches, axis=1)
patch_medians.shape

(20000, 13)

In [37]:
%%timeit -n 1 -r 3
# get classifications from fmask
with rasterio.open('/data/mbp14mtp/devon/devon_cloudmask.img') as src:
    # resolution of cloudmask is only 20m as fmask was taking forever at 10m
    img = np.repeat(np.repeat(src.read(1), 2, axis=1), 2, axis=0)
    y = extract_patches_2d(img, (3, 3),
                           max_patches=20000,
                           random_state=seed)
    y = y.reshape((20000, 9))
    y = mode(y, axis=1)[0].reshape(20000)

1 loop, best of 3: 517 ms per loop


In [38]:
X_train, X_test, y_train, y_test = train_test_split(
    patch_medians, y, test_size=0.05, random_state=1010
)

In [47]:
%%timeit -n 1 -r 1
rf = RandomForestClassifier(n_estimators=1000)
rf = rf.fit(X=X_train, y=y_train)

1 loop, best of 1: 54.6 s per loop


In [46]:
rf.score(X_test, y_test)

0.83699999999999997

RF does well with the cloud, water and clear. There is not really any snow in the image. Unsuprisingly the model does not predict shade very well, I think this is because the Fmask algorithm predicts where shade should be from the angles of the satellite and the sun, rather than from the spectral data directly, or a mixture of both. Shadow is also strongly buffered in the output, so may actually contain a large amount of clear land/water

In [44]:
# order is: clear, cloud, shadow, snow, water
confusion_matrix(rf.predict(X_test), y_test)

array([[527,  22,  66,   0,   1],
       [  2,  68,   4,   1,   3],
       [ 14,   4,  42,   0,   6],
       [  0,   0,   0,   0,   0],
       [  0,   2,  38,   0, 200]])

RF classifiers may be a viable option, but the real test would be to see if a classifier trained on one image or set of images, can successfully classify cloud in a totally new image.